In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

%config Completer.use_jedi = False

In [2]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
mask_detection = tf.keras.models.load_model("mask_detection_vibi67_100.h5")

In [3]:
text_mask = "Mask On"
text_no_mask = "Mask Off"
font = cv2.FONT_HERSHEY_SIMPLEX # font for displaying text (below)
scale = 0.8

In [4]:
def predict(image): # image is a numpy array
    
    face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    face_frame = cv2.resize(face_frame, (224, 224))
    face_frame = img_to_array(face_frame) # convert to numpy array
    face_frame = np.expand_dims(face_frame, axis=0) # expand dimension
    face_frame =  preprocess_input(face_frame) # preprocess image
    prediction = mask_detection.predict(face_frame)
    
    return prediction[0][0] # return probability of mask

In [5]:
def detector(gray_image, frame): # gray_image is a numpy array
    
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5) # detect faces
    
    for (x,y,w,h) in faces:

        roi_color = frame[y:y+h, x:x+w] # region of interest
        mask = predict(roi_color)

        
        if mask > 0.5: # if mask is on
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, text =text_mask, org =(x+50,y-100), fontFace =font, fontScale = scale, color =(0,255,0),
            thickness = 2)
        
        elif mask<=0.5: # if mask is off
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
            cv2.putText(frame, text =text_no_mask, org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),
            thickness = 2)
            
    return frame

In [ ]:
video_cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_cap.read() # read frame from video
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    detect = detector(gray_frame, frame)
    
    cv2.imshow("Video", detect) # display
    
    if cv2.waitKey(1) & 0xFF == ord("q"): # press q to quit
        break 
        
video_cap.release()
cv2.destroyAllWindows()